# Посчитать числа для сводной таблицы

In [1]:
import pandas as pd
import numpy as np

directory = 'data'

df = pd.read_csv(directory + '/cleared_dataset.csv')
df

/var/folders/bh/d932d5zj56z9_y_7lh3s66ndf1n4c9/T/ipykernel_37814/1159173139.py:6: DtypeWarning: Columns (34) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(directory + '/cleared_dataset.csv')


,Дата,ДатаДоставки,НомерЗаказаНаСайте,НовыйСтатус,СуммаЗаказаНаСайте,СуммаДокумента,МетодДоставки,ФормаОплаты,Регион,Группа2,...,НомерСтроки,КоличествоПроданоКлиенту,ДатаЗаказаНаСайте,Телефон_new,ЭлектроннаяПочта_new,Клиент,ID_SKU,ГородМагазина,МагазинЗаказа,МесяцДатыЗаказа
0,01.01.2017 17:15,04.01.2017,3530280_TR,Возврат из ПВЗ,2800.0,2800.0,Магазины,Безналичная,Москва,"ТЕКСТИЛЬ, ТРИКОТАЖ",...,2.0,0,01.01.2017,55575454-49504949555170,110109_nm19@gmail.com,Анжелика,IDL00023865048,0,-,1
1,01.01.2017 17:15,04.01.2017,3530280_TR,Возврат из ПВЗ,2800.0,2800.0,Магазины,Безналичная,Москва,"ТЕКСТИЛЬ, ТРИКОТАЖ",...,1.0,0,01.01.2017,55575454-49504949555170,110109_nm19@gmail.com,Анжелика,IDL00022519452,0,-,1
2,01.01.2017 17:38,03.01.2017,3530303_TR,К отгрузке,3585.0,3585.0,Курьерская,Наличная,Москва,ПОДГУЗНИКИ,...,1.0,0,01.01.2017,55574851-55555555555176,49117_1u16@mail.ru,Anastasia,IDL00038454553,0,-,1
3,01.01.2017 17:38,04.01.2017,3530318_TR,Доставлен,2510.0,2510.0,Магазины,Безналичная,Москва,"ТЕКСТИЛЬ, ТРИКОТАЖ",...,1.0,1,02.01.2017,55575054-56575354545171,115117_su21@mail.ru,Гость,IDL00035301654,0,-,1
4,01.01.2017 17:38,02.01.2017,3530313_TR,К отгрузке,1069.0,1069.0,Самовывоз,Наличная,Санкт-Петербург,ИГРУШКИ,...,1.0,1,02.01.2017,55575149-50524951545076,106117_ju18@yandex.ru,Гость,ID9010025983048,0,-,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4326294,24.01.2018 16:10,26.01.2018,7197353_TR,Доставлен,4029.0,3125.0,Магазины,Безналичная,Химки (Московская область район),ИГРУШКИ,...,1.0,1,24.01.2018,55574957-49485057515771,105117_iu24@mail.ru,Ирина,IDL00002922149,0,-,1
4326295,23.01.2018 17:42,31.01.2018,7189958_TR,Принят в ПВЗ,1645.0,1645.0,Магазины,Безналичная,Челябинск,ИГРУШКИ,...,17.0,1,23.01.2018,55574856-53565257495272,108117_lu17@mail.ru,Любовь,IDL00002922149,0,-,1
4326296,23.01.2018 23:56,26.01.2018,7196790_TR,К отгрузке,2095.0,2095.0,Pick point,Безналичная,Домодедово (Домодедовский район),ИГРУШКИ,...,4.0,1,23.01.2018,55574851-53575255565677,guest_103117_gu28@workmail.ru,Гость,IDL00002922149,0,-,1
4326297,23.01.2018 22:56,31.01.2018,7195888_TR,Доставлен,1519.0,1519.0,Магазины,Безналичная,Липецк,ИГРУШКИ,...,2.0,1,23.01.2018,55574852-50574855555171,118117_vu15@mail.ru,Анна,IDL00002922149,0,-,1


In [8]:
df['Цена x Количество'] = df['Цена'] * df['Количество']

In [9]:
from tqdm import tqdm

sold_data = []
placed_data = []
months_names = ['Янв', 'Фев', 'Март', 'Апр', 'Май', 'Июнь', 'Июль', 'Август', 'Сентябрь', 'Октябрь', 'Ноябрь', 'Декабрь']
for i in tqdm(range(1, 13)):
    placed = df[df['МесяцДатыЗаказа'] == i][df['ГодДатыЗаказа'] == '2 017']
    
    revenue = (placed['Количество'] * placed['Цена']).sum()
    major = placed['Маржа'].sum()
    count = placed['Количество'].sum()
    unique_orders = placed['НомерЗаказаНаСайте'].nunique()
    unique_clients = placed['Телефон_new'].nunique()
    average_order_revenue = placed[['Цена x Количество', 'НомерЗаказаНаСайте']].groupby(['НомерЗаказаНаСайте']).sum().mean()
    major_per_product = major / count
    major_percent = 100.0 * major / revenue
    average_number_of_products = placed[['Количество', 'НомерЗаказаНаСайте']].groupby(['НомерЗаказаНаСайте']).sum().mean()
    placed_data.append(
        {
            'alias': f'{months_names[i - 1]} 2017',
            'revenue': round(float(revenue), 0),
            'major': round(float(major), 0),
            'count': round(int(count), 0),
            'unique_orders': unique_orders,
            'unique_clients': unique_clients,
            'average_order_revenue': round(float(average_order_revenue.iloc[0]), 1),
            'major_per_product': round(float(major_per_product), 1),
            'major_percent': round(float(major_percent), 1),
            'average_number_of_products': round(float(average_number_of_products.iloc[0]), 1)
        }
    )

    
    sold = df[df['МесяцДатыЗаказа'] == i][df['ГодДатыЗаказа'] == '2 017'][df['Статус'] == 'Доставлен'][df['НовыйСтатус'] == 'Доставлен']

    revenue = (sold['Количество'] * sold['Цена']).sum()
    major = sold['Маржа'].sum()
    count = sold['Количество'].sum()
    unique_orders = sold['НомерЗаказаНаСайте'].nunique()
    unique_clients = sold['Телефон_new'].nunique()
    average_order_revenue = sold[['Цена x Количество', 'НомерЗаказаНаСайте']].groupby(['НомерЗаказаНаСайте']).sum().mean()
    major_per_product = major / count
    major_percent = 100.0 * major / revenue
    average_number_of_products = sold[['Количество', 'НомерЗаказаНаСайте']].groupby(['НомерЗаказаНаСайте']).sum().mean()
    sold_data.append(
        {
            'alias': f'{months_names[i - 1]} 2017',
            'revenue': round(float(revenue), 0),
            'major': round(float(major), 0),
            'count': round(int(count), 0),
            'unique_orders': unique_orders,
            'unique_clients': unique_clients,
            'average_order_revenue': round(float(average_order_revenue.iloc[0]), 1),
            'major_per_product': round(float(major_per_product), 1),
            'major_percent': round(float(major_percent), 1),
            'average_number_of_products': round(float(average_number_of_products.iloc[0]), 1)
        }
    )


for i in tqdm(range(1, 3)):
    placed = df[df['МесяцДатыЗаказа'] == i][df['ГодДатыЗаказа'] == '2 018']
    
    revenue = (placed['Количество'] * placed['Цена']).sum()
    major = placed['Маржа'].sum()
    count = placed['Количество'].sum()
    unique_orders = placed['НомерЗаказаНаСайте'].nunique()
    unique_clients = placed['Телефон_new'].nunique()
    average_order_revenue = placed[['Цена x Количество', 'НомерЗаказаНаСайте']].groupby(['НомерЗаказаНаСайте']).sum().mean()
    major_per_product = major / count
    major_percent = 100.0 * major / revenue
    average_number_of_products = placed[['Количество', 'НомерЗаказаНаСайте']].groupby(['НомерЗаказаНаСайте']).sum().mean()
    placed_data.append(
        {
            'alias': f'{months_names[i - 1]} 2018',
            'revenue': round(float(revenue), 0),
            'major': round(float(major), 0),
            'count': round(int(count), 0),
            'unique_orders': unique_orders,
            'unique_clients': unique_clients,
            'average_order_revenue': round(float(average_order_revenue.iloc[0]), 1),
            'major_per_product': round(float(major_per_product), 1),
            'major_percent': round(float(major_percent), 1),
            'average_number_of_products': round(float(average_number_of_products.iloc[0]), 1)
        }
    )

    
    sold = df[df['МесяцДатыЗаказа'] == i][df['ГодДатыЗаказа'] == '2 018'][df['Статус'] == 'Доставлен'][df['НовыйСтатус'] == 'Доставлен']

    revenue = (sold['Количество'] * sold['Цена']).sum()
    major = sold['Маржа'].sum()
    count = sold['Количество'].sum()
    unique_orders = sold['НомерЗаказаНаСайте'].nunique()
    unique_clients = sold['Телефон_new'].nunique()
    average_order_revenue = sold[['Цена x Количество', 'НомерЗаказаНаСайте']].groupby(['НомерЗаказаНаСайте']).sum().mean()
    major_per_product = major / count
    major_percent = 100.0 * major / revenue
    average_number_of_products = sold[['Количество', 'НомерЗаказаНаСайте']].groupby(['НомерЗаказаНаСайте']).sum().mean()
    sold_data.append(
        {
            'alias': f'{months_names[i - 1]} 2018',
            'revenue': round(float(revenue), 0),
            'major': round(float(major), 0),
            'count': round(int(count), 0),
            'unique_orders': unique_orders,
            'unique_clients': unique_clients,
            'average_order_revenue': round(float(average_order_revenue.iloc[0]), 1),
            'major_per_product': round(float(major_per_product), 1),
            'major_percent': round(float(major_percent), 1),
            'average_number_of_products': round(float(average_number_of_products.iloc[0]), 1)
        }
    )

/var/folders/bh/d932d5zj56z9_y_7lh3s66ndf1n4c9/T/ipykernel_37814/2247295644.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.it/s]
  placed = df[df['МесяцДатыЗаказа'] == i][df['ГодДатыЗаказа'] == '2 017']
/var/folders/bh/d932d5zj56z9_y_7lh3s66ndf1n4c9/T/ipykernel_37814/2247295644.py:34: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  sold = df[df['МесяцДатыЗаказа'] == i][df['ГодДатыЗаказа'] == '2 017'][df['Статус'] == 'Доставлен'][df['НовыйСтатус'] == 'Доставлен']
/var/folders/bh/d932d5zj56z9_y_7lh3s66ndf1n4c9/T/ipykernel_37814/2247295644.py:34: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  sold = df[df['МесяцДатыЗаказа'] == i][df['ГодДатыЗаказа'] == '2 017'][df['Статус'] == 'Доставлен'][df['НовыйСтатус'] == 'Доставлен']
/var/folders/bh/d932d5zj56z9_y_7lh3s66ndf1n4c9/T/ipykernel_37814/2247295644.py:34: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  sold = df[df['

In [11]:
print(placed_data)

[{'alias': 'Янв 2017', 'revenue': 182272493.0, 'major': 35912008.0, 'count': 307702, 'unique_orders': 72645, 'unique_clients': 54448, 'average_order_revenue': 2509.1, 'major_per_product': 116.7, 'major_percent': 19.7, 'average_number_of_products': 4.2}, {'alias': 'Фев 2017', 'revenue': 156075993.0, 'major': 31242621.0, 'count': 273989, 'unique_orders': 66785, 'unique_clients': 50819, 'average_order_revenue': 2337.0, 'major_per_product': 114.0, 'major_percent': 20.0, 'average_number_of_products': 4.1}, {'alias': 'Март 2017', 'revenue': 205740473.0, 'major': 43305776.0, 'count': 370006, 'unique_orders': 84446, 'unique_clients': 61590, 'average_order_revenue': 2436.4, 'major_per_product': 117.0, 'major_percent': 21.0, 'average_number_of_products': 4.4}, {'alias': 'Апр 2017', 'revenue': 194512652.0, 'major': 44508661.0, 'count': 377152, 'unique_orders': 80758, 'unique_clients': 58871, 'average_order_revenue': 2408.6, 'major_per_product': 118.0, 'major_percent': 22.9, 'average_number_of_pro

In [15]:
print(placed_data[0].keys())
print(sold_data[0].keys())

dict_keys(['alias', 'revenue', 'major', 'count', 'unique_orders', 'unique_clients', 'average_order_revenue', 'major_per_product', 'major_percent', 'average_number_of_products'])
dict_keys(['alias', 'revenue', 'major', 'count', 'unique_orders', 'unique_clients', 'average_order_revenue', 'major_per_product', 'major_percent', 'average_number_of_products'])


# Сохранить статистику

In [18]:
import csv

statistics_dir = 'statistics'

with open(statistics_dir + '/placed_data.csv', 'w', newline='') as csvfile:
    fieldnames = placed_data[0].keys()
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for row in placed_data:
        writer.writerow(row)

        
with open(statistics_dir + '/sold_data.csv', 'w', newline='') as csvfile:
    fieldnames = sold_data[0].keys()
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for row in sold_data:
        writer.writerow(row)